# Using Deep Convolutional Q Learning Artificial Neural Network
###### (the more words we put, the cooler it sounds)
## Group 1, Bot Bot
## Group Members: Mohammed Alnasser, Jesus Nu ̃nez, Ankit Dhingra

# Double Deep Q-Network

In 2016, Google DeepMind ([Link](https://www.aaai.org/ocs/index.php/AAAI/AAAI16/paper/download/12389/11847)) decided to alter the DQN algorithm the same way the original Q-Learner algorithm was updated by adding a second network. The team found that the overestimation that affected the q-learner also affected the DQN algoritm. When the original double q-learner was introduced they proved that it worked in that setting and in this paper they prove it can be generalized to work with large scale function approximation.

**Summary**  
This algorithm combines the benefits of the Double Q-Learner as the benefits of deep learning. We get the function approximation so that we can have a continuous state space plus we keep from having the overestimation from Q-Learning. This document will be a combination of the DQN and Double Q-Learner so it will be mostly review. But, this algorithm is so much more powerful that you should explore the other gyms at OpenAI and see what you can solve.  

One thing to note, we need to update the weights of the second neural network with the weights from the first neural network. We didn't do this in the Double Q-Learner since the tables were both getting updated.

**CartPole Example**  
Again we will use the [CartPole](https://gym.openai.com/envs/CartPole-v1/) environment from OpenAI.  

The actions are 0 to push the cart to the left and 1 to push the cart to the right.  

The continuous state space is an X coordinate for location, the velocity of the cart, the angle of the pole, and the velocity at the tip of the pole. The X coordinate goes from -4.8 to +4.8, velocity is -Inf to +Inf, angle of the pole goes from -24 degrees to +24 degrees, tip velocity is -Inf to +Inf. With all of the possible combinations you can see why we can't create a Q table for each one.  

To "solve" this puzzle you have to have an average reward of > 195 over 100 consecutive episodes. One thing to note, I am hard capping the rewards at 210 so this number can't average above that and it also could potentially drive the average down.

In [9]:
from Engine import Blockudoku
import pygame as pg
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPool2D

from time import sleep
from IPython.display import clear_output

import random

#Create game
env = Blockudoku()

**Double Deep Q-Network Class**  
This class is the same as the DQN class from the last notebook with a few exceptions.  
**init**:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;We create a second NN for the target network  

**update_target_from_model(self)**  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;This class updates the weights of the target NN from the model NN

**build_model(self)**:  
**action(self,state)**:  
**test_action(self,state)**:  
**store(self, state, action, reward, nstate, done)**:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Same  

**experience_replay(self, batch_size)**:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;This class has the Double DQN changes. We grab the prediction targets from the target NN and then use that in the Q update rule.    

In [2]:
class DoubleDeepQNetwork():
    def __init__(self, states, actions, alpha, gamma, epsilon,epsilon_min, epsilon_decay):
        self.nS = states
        self.nA = actions
        self.memory = deque([], maxlen=2500)
        self.alpha = alpha
        self.gamma = gamma
        #Explore/Exploit
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.model = self.build_model()
        self.model_target = self.build_model() #Second (target) neural network
        self.update_target_from_model() #Update weights
        self.loss = []
        
    def build_model(self):
        cnn_layers = [Input(shape=self.nS),
                      Conv2D(16, 3, activation="relu", padding="same" , name="Conv2D_layer1"),
                      MaxPool2D(),
                      Conv2D(32, 3, activation="relu", padding="same", name="Conv2D_layer2"),
                      MaxPool2D(),
                      Dense(69, activation="relu", name="Dense_layer1"),
                      Dense(69, activation="relu", name="Dense_layer2"),
                      Flatten(),
                      
                      Dense(self.nA, activation="linear", name="output")]
        model = keras.Sequential(cnn_layers)
        
        model.compile(loss='mse', optimizer="adam")
        return model

    def update_target_from_model(self):
        #Update the target model from the base model
        self.model_target.set_weights( self.model.get_weights() )

    def action(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.nA) #Explore
        action_vals = self.model.predict(state) #Exploit: Use the NN to predict the correct action from this state
        return np.argmax(action_vals[0])

    def test_action(self, state): #Exploit
        action_vals = self.model.predict(state)
        return np.argmax(action_vals[0])

    def store(self, state, action, reward, nstate, done):
        #Store the experience in memory
        self.memory.append( (state, action, reward, nstate, done) )

    def experience_replay(self, batch_size):
        #Execute the experience replay
        minibatch = random.sample( self.memory, batch_size) #Randomly sample from memory

        x = []
        y = []
        st = np.zeros((0, self.nS[0], self.nS[1], self.nS[2])) #States
        nst = np.zeros( (0, self.nS[0], self.nS[1], self.nS[2]) )#Next States
        for i in range(len(minibatch)): #Creating the state and next state np arrays
            st = np.append( st, minibatch[i][0], axis=0)
            nst = np.append( nst, minibatch[i][3], axis=0)
        st_predict = self.model.predict(st) #Here is the speedup! I can predict on the ENTIRE batch
        nst_predict = self.model.predict(nst)
        nst_predict_target = self.model_target.predict(nst) #Predict from the TARGET
        index = 0
        for state, action, reward, nstate, done in minibatch:
            x.append(state)
            #Predict from state
            nst_action_predict_target = nst_predict_target[index]
            nst_action_predict_model = nst_predict[index]
            if done == True: #Terminal: Just assign reward much like {* (not done) - QB[state][action]}
                target = reward
            else:   #Non terminal
                target = reward + self.gamma * nst_action_predict_target[np.argmax(nst_action_predict_model)] 
            target_f = st_predict[index]
            target_f[action] = target
            y.append(target_f)
            index += 1
        #Reshape for Keras Fit
        x_reshape = np.array(x).reshape((batch_size, self.nS[0], self.nS[1], self.nS[2]))
        y_reshape = np.array(y).reshape((batch_size, self.nA))
        epoch_count = 1
        hist = self.model.fit(x_reshape, y_reshape, epochs=epoch_count, verbose=0)
        #Graph Losses
        for i in range(epoch_count):
            self.loss.append( hist.history['loss'][i] )
        #Decay Epsilon
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [3]:
EPISODES = 500
TRAIN_END = 0
discount_rate = 0.3  # Gamma
learning_rate = 0.001  # Alpha
batch_size = 24

nS = env.state.shape
nA = env.action_space.n #Actions


In [4]:
# pg.quit()

In [5]:
dqn = DoubleDeepQNetwork(nS, nA, learning_rate, discount_rate, 1, 0.001, 0.9995)

In [20]:
render_pygame = True
render = True

# start pygame
if render_pygame:
    pg.init()
    screen = pg.display.set_mode([env.window_size.x, env.window_size.y])
    env.setScreen(screen)
env.reset()

#Training
rewards = [] #Store rewards for graphing
epsilons = [] # Store the Explore/Exploit
scores = []
actions = []
running = True
for e in range(EPISODES):
    state = env.reset()
    state = np.reshape(state, [1, nS[0], nS[1], nS[2]]) # Resize to store in memory to pass to .predict
    tot_rewards = 0
    time = 0
    
    for step in range(100):
        running = env.drawGameHeadless()
        
        if not running:
            break
        
        action = dqn.action(state)
        nstate, reward, done, _ = env.step(action)
        nstate = np.reshape(state, [1, nS[0], nS[1], nS[2]])
        tot_rewards += reward
        dqn.store(state, action, reward, nstate, done)
        state = nstate
        
        if render:
            clear_output(wait=True)
            print(f"Episode: {e + 1}")
            env.render()
            print(f"Step: {step}")
            print(f"Action: {action}")
            print(f"Reward: {reward}")
            print(f"Total rewards: {tot_rewards}")
            sleep(.2)
        
        if done or step == 99:
            rewards.append(tot_rewards)
            epsilons.append(dqn.epsilon)
            scores.append(env.score)
            actions.append(time)
            print("episode: {}/{}, score: {}, actions: {}, e: {}"
                  .format(e, EPISODES, env.score, time, dqn.epsilon))
            break
        #Experience Replay
        if len(dqn.memory) > batch_size:
            dqn.experience_replay(batch_size)
        
        time += 1
            
    if not running:
        break
    #Update the weights after each episode (You can configure this for x steps as well
    dqn.update_target_from_model()
    #If our current NN passes we are done
    #I am going to use the last 5 runs
    if len(rewards) > 5 and np.average(rewards[-5:]) > 195:
        #Set the rest of the EPISODES for testing
        TEST_Episodes = EPISODES - e
        TRAIN_END = e
        break

pg.quit()

Episode: 2
+-----+-----+-----+
| : : | : : | : : |
| : : | : : | : : |
| : : | : : | : : |
+-----+-----+-----+
| : : | : : | : : |
| : : | : : | : : |
| : : | : : | : : |
+-----+-----+-----+
| : : | : : | : : |
| : : | : : | : : |
| : : | : : | : : |
+-----+-----+-----+
Step: 70
Action: 2
Reward: -1
Total rewards: -162


In [ ]:
#Testing
print('Training complete. Testing started...')
#TEST Time
#   In this section we ALWAYS use exploit don't train any more

render_pygame = True
render = True

if render_pygame:
    pg.init()
    screen = pg.display.set_mode([env.window_size.x, env.window_size.y])
    env.setScreen(screen)
env.reset()

TEST_Episodes = 3
running = True
for e_test in range(TEST_Episodes):
    state = env.reset()
    state = np.reshape(state, [1, nS[0], nS[1], nS[2]])
    tot_rewards = 0
    for step in range(100):
        if render_pygame:
            running = env.drawGameHeadless()
                
        if not running:
            break
        
        action = dqn.test_action(state)
        nstate, reward, done, _ = env.step(action)
        nstate = np.reshape(state, [1, nS[0], nS[1], nS[2]])
        tot_rewards += reward
        #DON'T STORE ANYTHING DURING TESTING
        state = nstate
        
        if render:
            clear_output(wait=True)
            print(f"Episode: {e + 1}")
            env.render()
            print(f"Step: {step}")
            print(f"Action: {action}")
            print(f"Reward: {reward}")
            print(f"Total rewards: {tot_rewards}")
            sleep(.2)
        
        if done or step == 99: 
            rewards.append(tot_rewards)
            epsilons.append(0) #We are doing full exploit
            print("episode: {}/{}, score: {}, actions: {}, e: {}"
                  .format(e, EPISODES, env.score, time, 0))
            break
    
    if not running:
            break
    
pg.quit()

**Results**  
Here is a graph of the results. If everything was done correctly you should see the rewards over the red line.  

Black: This is the 100 episode rolling average  
Red: This is the "solved" line at 195  
Blue: This is the reward for each episode  
Green: This is the value of epsilon scaled by 200  
Yellow: This is where the tests started.

In [ ]:
#Plotting
rolling_average = np.convolve(rewards, np.ones(100)/100)

plt.plot(rewards)
plt.plot(rolling_average, color='black')
plt.axhline(y=195, color='r', linestyle='-') #Solved Line
#Scale Epsilon (0.001 - 1.0) to match reward (0 - 200) range
eps_graph = [200*x for x in epsilons]
plt.plot(eps_graph, color='g', linestyle='-')
#Plot the line where TESTING begins
plt.axvline(x=TRAIN_END, color='y', linestyle='-')
plt.xlim( (0,EPISODES) )
plt.ylim( (-1200,220) )
plt.show()

**Changes**  
These are all the same changes as the DQN notebook with the exception of the update weights parameter.  
*hyper parameters*: You can alter alpha, gamma, batch size, and episode length to see what differences the algorithm returns.  
*Training End*: You can also change the line where I only check the last 5 runs before switching to testing mode (if len(rewards) > 5 and np.average(rewards[-5:]) > 195:) as that doesn't prove it was solved. The reason I did this was because I wanted to limit the amount of runs I made.  
*Update Weights*: I call 'dqn.update_target_from_model()' after every episode. You can adjust this to run at different times. I have done per step (no matter how long the episode ran) and I have seen it done every X episodes. Feel free to try different things.

**Conclusion**  
This is a Double Deep Q-Network implementation. There are some changes you can make here and there but it follows the paper as close as I could. If you want to dive deeper you can see that the paper has graphs that dive deeper into the inner workings of the neural network.  

**Reference**  
Van Hasselt, H., Guez, A., & Silver, D. (2016, February). Deep Reinforcement Learning with Double Q-Learning. In AAAI (Vol. 2, p. 5)

## Project Conclusion  
This completes the set of notebooks that cover the original Q-Learner and continues through recent updates. Once you have worked in this area for a while you can see how powerful that first update statement really was. With just some slight tweaks, these algorithms were able to achieve higher scores than even advanced Atari players.  

I hope these notebooks have peaked your interest enough to continue your reinforcement journey.